In [2]:
import pandas as pd
import numpy as np

file_to_load = "Resources/purchase_data.csv"

purchase_data = pd.read_csv(file_to_load)

In [3]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
#Player Count

summary_players = pd.DataFrame({"Total Players": [purchase_data["SN"].nunique()]})
summary_players

,Total Players
0,576


In [5]:
#Purchasing Analysis (Total)

items = purchase_data["Item Name"].nunique()

price = purchase_data["Price"].mean()

number_of_purchases = purchase_data["Item Name"].count()

total_revenue = purchase_data["Price"].sum()

In [6]:
def format_as_currency(number):
  return "${:,.2f}".format(number)
purchasing_summary = pd.DataFrame({"Number of Unique Items": [items], "Average Price": [price], 
                       "Number of Purchases": [number_of_purchases], "Total Revenue": [total_revenue]})
purchasing_summary["Average Price"] = purchasing_summary['Average Price'].map(format_as_currency)
purchasing_summary["Total Revenue"] = purchasing_summary['Total Revenue'].map(format_as_currency)
purchasing_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [7]:
#Gender Demographics

gender = purchase_data.groupby(["Gender"])
unique_genders = gender.nunique()
list_drop = ['Purchase ID', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']
unique_genders.drop(list_drop, axis=1, inplace=True)

In [8]:
def format_as_percent(number):
  return "{:,.2f}%".format(number)
percent_players = unique_genders["SN"]/int(unique_genders.sum())*100
percentage_of_players = pd.DataFrame(percent_players.map(format_as_percent))

In [9]:
gender_demo = pd.merge(unique_genders, percentage_of_players, on="Gender")
gender_demo.rename(columns={"SN_x": "Total Count", "SN_y": "Percentage of Players"})

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [10]:
#Purchasing Analysis (Gender)

average_price_gender = gender["Price"].mean()

average = pd.DataFrame({"Average Price": average_price_gender.map(format_as_currency)})

total_purchase = gender["Price"].sum()
purchases = pd.DataFrame({"Total Purchase Value": total_purchase.map(format_as_currency)})

SN = gender.nunique()

total_purchase_person = total_purchase/SN["SN"]
totals = pd.DataFrame({"Avg Total Purchase per Person": total_purchase_person.map(format_as_currency)})

people_per_gender = SN["Purchase ID"]
people = pd.DataFrame({"Purchase Count": people_per_gender})

In [11]:
gender_purchasing = pd.merge(people, average, on="Gender")
gender_purchasing2 = pd.merge(purchases, totals, on="Gender")

all_gender_purchasing = pd.merge(gender_purchasing, gender_purchasing2, on="Gender")
all_gender_purchasing

,Purchase Count,Average Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [12]:
#Age Demographics

bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", 
               "35-39", "40+"]
purchase_data["Age Demographics"] = pd.cut(purchase_data["Age"], bins, labels=age_labels)

In [13]:
age_demos = purchase_data.groupby("Age Demographics")

age_count = pd.DataFrame(age_demos["Age"].count())

In [14]:
percents = age_count["Age"]/int(age_count.sum())*100

percent_of_ages = pd.DataFrame(percents.map(format_as_percent))

In [15]:
age_demographics = pd.merge(age_count, percent_of_ages, on="Age Demographics")
age_demographics.rename(columns={"Age_x": "Total Count", "Age_y": "Percentage of Players"})

,Total Count,Percentage of Players
Age Demographics,,
<10,23,2.95%
10-14,28,3.59%
15-19,136,17.44%
20-24,365,46.79%
25-29,101,12.95%
30-34,73,9.36%
35-39,41,5.26%
40+,13,1.67%


In [16]:
#Purchasing Analysis (Age)
average_price_age = age_demos["Price"].mean()
average_age = pd.DataFrame({"Average Purchase Price": average_price_age.map(format_as_currency)})

total_purchase_age = age_demos["Price"].sum()
purchases_age = pd.DataFrame({"Total Purchase Value": total_purchase_age.map(format_as_currency)})

SN_age = age_demos.nunique()
av_total_person = total_purchase_age/SN_age["SN"]
totals_age = pd.DataFrame({"Avg Total Purchase per Person": av_total_person.map(format_as_currency)})

people_per_age = SN_age["Purchase ID"]
people_age = pd.DataFrame({"Purchase Count": people_per_age})

In [17]:
age_purchasing = pd.merge(people_age, average_age, on="Age Demographics")
age_purchasing2 = pd.merge(purchases_age, totals_age, on="Age Demographics")

all_age_purchasing = pd.merge(age_purchasing, age_purchasing2, on="Age Demographics")
all_age_purchasing

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Demographics,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [18]:
# Top Spenders
screen_name = purchase_data.groupby(["SN"])
unique_SN = screen_name.nunique()
unique_SN.head()

drop_list = ['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']
unique_SN.drop(drop_list, axis=1, inplace=True)

In [19]:
average_price_SN = screen_name["Price"].mean()

average_SN = pd.DataFrame({"Average Price": average_price_SN.map(format_as_currency)})

total_purchase_SN = screen_name["Price"].sum()
purchases_SN = pd.DataFrame({"Total Purchase Value": total_purchase_SN.map(format_as_currency)})

names = screen_name.nunique()

total_purchase_per_SN = total_purchase_SN/names["SN"]
totals_SN = pd.DataFrame({"Avg Total Purchase per Person": total_purchase_per_SN.map(format_as_currency)})

people_per_SN = names["Purchase ID"]
people_SN = pd.DataFrame({"Purchase Count": people_per_SN})

In [20]:
SN_purchasing_per_total = people_SN.sort_values(["Purchase Count"], ascending=False)

SN_purchasing = pd.merge(SN_purchasing_per_total, average_SN, on="SN")

all_SN_purchasing = pd.merge(SN_purchasing, purchases_SN, on="SN")

all_SN_purchasing.head()

,Purchase Count,Average Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


In [21]:
item_name = purchase_data.groupby(["Item ID", "Item Name"])

unique_item = item_name.nunique()

drop_list_3 = ['SN', 'Age', 'Gender', 'Item Name', 'Item ID', 'Price']
unique_item.drop(drop_list_3, axis=1, inplace=True)

In [22]:
average_price_item = item_name["Price"].mean()
average_item = pd.DataFrame({"Average Price": average_price_item.map(format_as_currency)})

total_purchase_item = item_name["Price"].sum()
purchases_item = pd.DataFrame({"Total Purchase Value": total_purchase_item.map(format_as_currency)})

item_names = item_name.nunique()

total_purchase_per_item = total_purchase_item/item_names["SN"]
totals_item = pd.DataFrame({"Avg Total Purchase per Person": total_purchase_per_item.map(format_as_currency)})

people_per_item = unique_item["Purchase ID"]
people_item = pd.DataFrame({"Purchase Count": people_per_item})

In [23]:
item_purchasing_per_total = people_item.sort_values(["Purchase Count"], ascending=False)

item_purchasing = pd.merge(item_purchasing_per_total, average_item, how='left', on=["Item ID", "Item Name"])

all_item_purchasing = pd.merge(item_purchasing, purchases_item, how='left', on=["Item ID", "Item Name"])

all_item_purchasing.head()

,,Purchase Count,Average Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [24]:
item_purchasing_2 = pd.merge(people_item, average_item, how='left', on=["Item ID", "Item Name"])

total_value = purchases_item.sort_values(["Total Purchase Value"], ascending=False)

all_item_purchasing_2 = pd.merge(item_purchasing_2, item_purchasing_total_value, 
                                 how='left', on=["Item ID", "Item Name"])

all_item_purchasing_2.head()

NameError: name 'item_purchasing_total_value' is not defined